# 📓 TruLens with Outside Logs in a Dataframe

If your application was run (and logged) outside of TruLens, TruVirtual can be used to ingest and evaluate the logs.

This notebook walks through how to quickly log a dataframe of prompts, responses and contexts (optional) to TruLens as traces, and how to run evaluations with the trace data.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/examples/quickstart/add_dataframe_quickstart.ipynb)

In [ ]:
# !pip install trulens trulens-providers-openai openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
from trulens.apps.virtual import VirtualApp
from trulens.core import Select

virtual_app = VirtualApp()

Next, let's define feedback functions.

The `add_dataframe` method will load the prompt, context and response into virtual records. We should define our feedback functions to access this data in the structure it will be stored. We can do so as follows:

- prompt: selected using `.on_input()`
- response: selected using `on_output()`
- context: selected using `Select.RecordCalls.retriever.get_context.rets[:]`

In [ ]:
from trulens.core import Feedback
from trulens.providers.openai import OpenAI

# Initialize provider class
provider = OpenAI()

# Select context to be used in feedback. We select the return values of the
# virtual `get_context` call in the virtual `retriever` component. This exact path
# should be used accessing context ingested using the `add_dataframe` method.
context = Select.RecordCalls.retriever.get_context.rets[:]

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(context)
)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()

## Start a TruLens logging session

In [ ]:
from trulens.core import TruSession
from trulens.dashboard import run_dashboard

session = TruSession()
session.reset_database()
run_dashboard(session)

## Set up the virtual recorder

In [ ]:
from trulens.apps.virtual import TruVirtual

virtual_recorder = TruVirtual(
    app_name="a virtual app",
    app=virtual_app,
    feedbacks=[f_context_relevance, f_groundedness, f_qa_relevance],
)

## Create or load a dataframe

The dataframe should include minimally columns named `prompt` and `response`. You can also include a column named `context` if you wish to evaluate retrieval systems or RAGs.

In [ ]:
import pandas as pd

data = {
    "prompt": ["Where is Germany?", "What is the capital of France?"],
    "response": ["Germany is in Europe", "The capital of France is Paris"],
    "context": [
        "Germany is a country located in Europe.",
        [
            "France is a country in Europe and its capital is Paris.",
            "Germanh is a country located in Europe",
        ],
    ],
}
df = pd.DataFrame(data)
df.head()

## Add the dataframe to TruLens

We can then add the dataframe to TruLens using the virual recorder method `add_dataframe`. Doing so will immediately log the traces, and kick off the computation of evaluations. After some time, the evaluation results will be accessible both from the sdk (e.g. `session.get_leaderboard`) and in the TruLens dashboard.

If you wish to skip evaluations and only log traces, you can simply skip the sections of this notebook where feedback functions are defined, and exclude them from the construction of the `virtual_recorder`.

In [ ]:
virtual_records = virtual_recorder.add_dataframe(df)